Questions for Jose: 
- what are we going to do with the empty sets (tweets not assigned to topics)?
    -not classifying, we put into a big context vector
- how are we going to compare the QA models to the real answers ?
- after finish the prototype: manually take Youtube data . youtube, no api search available, manually finding videos, might not be as good as twitter
- what to do when QA doesn't give answers, like in the examples


-empty sets are not the focus, optimisation for future work
- 1. further explore model
- then 2ways, more politicians in the same party, or increase data sources of politicians
- once we add all the topics to carlo and finish protoype, then meet with Luis
- mention in thesis, that odd number, you be neutral
- 100 tweets, manually label the answers to compare answers with QA answer, F1 score, evaluation, https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html, perhaps use binary (0-1), or 1-3 etc
- once prototype

To do: 
- 1. simplify questions
- 2. run algorithms on simplifed questions
- 3. label own answers and algorithms answer with positive or negative (0 or 1) 
- 4. evaluate the results with F1
- 
-find another language detector model
- simplify english wordings, check results using F1, then meet with Luis and Jose 

Writing: 
- simple past (passive), reffering particular study, no 'I' and 'we' 
- no page limit, avg 30pages



# Install and import libraries

In [1]:
!pip install pandas
!pip install tweepy
!pip install pycld2
!pip install tqdm
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [91]:
import tweepy
import pandas as pd
import re
import pycld2 as cld2
from transformers import pipeline
from tqdm import tqdm

# Get the tweets

In [92]:
#run keys notebook
%run ./twitter_keys.ipynb

In [93]:
# OAuth process, using the keys and tokens
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)
auth.set_access_token(access_token, access_token_secret)

In [94]:
#initialise lists for future df
tweets =[]
likes= []
time=[]
politician_id=[]

In [95]:
for i in tweepy.Cursor(api.user_timeline, id="CarloSommaruga", tweet_mode="extended").items():
    tweets.append(i.full_text)
    likes.append(i.favorite_count)
    time.append(i.created_at)
    politician_id.append(i.user.screen_name)

Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id
Unexpected parameter: id


In [293]:
df = pd.DataFrame({"tweets": tweets, "likes": likes, "time":time, "politician_id":politician_id})

In [309]:
df

,Unnamed: 0,tweets,likes,time,politician_id
0,0,Alors que le Conseil fédéral annonce des défic...,5,2022-09-14 08:08:53+00:00,CarloSommaruga
1,8,@HenrietteLevy Les excédents du fonds AVS pour...,0,2022-08-26 12:28:02+00:00,CarloSommaruga
2,9,"@feusl Es ist pikant, dass ein Fakejournalist,...",4,2022-08-26 12:22:12+00:00,CarloSommaruga
3,10,https://t.co/KeNl40pyix,13,2022-08-26 11:46:20+00:00,CarloSommaruga
4,11,@nashtags @lombardi1956 @muheim @Rustichelli74...,2,2022-08-22 05:44:51+00:00,CarloSommaruga
...,...,...,...,...,...
1812,3194,Comment le premier parti de suisse peut-il pro...,2,2015-12-08 17:13:38+00:00,CarloSommaruga
1813,3196,CH-EU:73% des entreprises craignent que résili...,0,2015-12-08 10:03:12+00:00,CarloSommaruga
1814,3197,"Suisse: De 2009 à 2014, le nombre de jeunes pr...",0,2015-12-07 08:51:27+00:00,CarloSommaruga
1815,3199,Suisse-UE: Clause de sauvegarde unilatérale: P...,1,2015-12-05 15:58:37+00:00,CarloSommaruga


In [295]:
df.to_csv("carlo_dataset.csv")

# Cleaning

In [5]:
df = pd.read_csv("./csv/carlo_dataset.csv")

In [6]:
#remove tweets from df 
df = df[~df.tweets.str.contains("RT")]
#reset index
df = df.reset_index(drop=True)

In [7]:
df

,Unnamed: 0,tweets,likes,time,politician_id
0,0,Alors que le Conseil fédéral annonce des défic...,5,2022-09-14 08:08:53+00:00,CarloSommaruga
1,8,@HenrietteLevy Les excédents du fonds AVS pour...,0,2022-08-26 12:28:02+00:00,CarloSommaruga
2,9,"@feusl Es ist pikant, dass ein Fakejournalist,...",4,2022-08-26 12:22:12+00:00,CarloSommaruga
3,10,https://t.co/KeNl40pyix,13,2022-08-26 11:46:20+00:00,CarloSommaruga
4,11,@nashtags @lombardi1956 @muheim @Rustichelli74...,2,2022-08-22 05:44:51+00:00,CarloSommaruga
...,...,...,...,...,...
1812,3194,Comment le premier parti de suisse peut-il pro...,2,2015-12-08 17:13:38+00:00,CarloSommaruga
1813,3196,CH-EU:73% des entreprises craignent que résili...,0,2015-12-08 10:03:12+00:00,CarloSommaruga
1814,3197,"Suisse: De 2009 à 2014, le nombre de jeunes pr...",0,2015-12-07 08:51:27+00:00,CarloSommaruga
1815,3199,Suisse-UE: Clause de sauvegarde unilatérale: P...,1,2015-12-05 15:58:37+00:00,CarloSommaruga


In [8]:
#cleaning function 

def cleaner(tweet):
    #remove urls
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    #remove mentions
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    #remove hash 
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    #remove emojis
    EMOJI_PATTERN = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\U0001f914-\U0001f914"
                           u"\U00002066-\U00002066"
                           "]+", flags=re.UNICODE)
    tweet = EMOJI_PATTERN.sub(r'', tweet)
    
    #return tweet
    return tweet
#apply cleaner to the function
df["tweets"] = df["tweets"].apply(cleaner)

In [10]:
#remove whitespaces
def remove_whitespaces(s):
    return s.strip()
df['tweets'] = df['tweets'].apply(remove_whitespaces)
#Convert NaN values to empty string
nan_value = float("NaN")
df.replace("", nan_value, inplace=True)
#Drop NaN
df.dropna(subset = ["tweets"], inplace=True)
#reset index
df = df.reset_index(drop=True)

# Language detection model using pycld2

In [14]:
# TODO: Find a different language detector!!!!!!!!!!

#initialise a list 
lang_detect = []

#append languages to list
for tweet in df["tweets"]:
    _, _, _, detected_language = cld2.detect(tweet,  returnVectors=True)
    lang_detect.append(detected_language)
    
#add languages to df
df["language"] = lang_detect

# Translate the Tweets

In [133]:
#translation model from french to english
model_checkpoint = "Helsinki-NLP/opus-mt-fr-en"
translator_fr = pipeline("translation", model=model_checkpoint)

#translation model from german to english
model_checkpoint = "Helsinki-NLP/opus-mt-de-en"
translator_de = pipeline("translation", model=model_checkpoint)

#translation model from italian to english
model_checkpoint = "Helsinki-NLP/opus-mt-it-en"
translator_it = pipeline("translation", model=model_checkpoint)

In [16]:
#initialise list
translated_tweets = []

#for loop for translation 
for i in tqdm(range(len(df))):
    if not df.iloc[i]['language']:
        translated_tweets.append('NA')
        continue
    
    if df.iloc[i]["language"][0][2] == "FRENCH":
        translated_tweets.append(translator_fr(df.iloc[i]["tweets"]))
    elif df.iloc[i]["language"][0][2] == "GERMAN":
        translated_tweets.append(translator_de(df.iloc[i]["tweets"]))
    elif df.iloc[i]["language"][0][2] == "ITALIAN":
        translated_tweets.append(translator_it(df.iloc[i]["tweets"]))
    else:
        translated_tweets.append("NA")
        
        

100%|██████████| 1805/1805 [23:09<00:00,  1.30it/s]


In [ ]:
#creating csv
# df_questions  = pd.read_csv('./csv/carlo_questions.csv')
# df_translated = pd.DataFrame({"translated_tweets": translated_tweets})
#df['translated_tweets'] = translated_tweets
#df.to_csv('df_translated_tweets.csv', index=False)

# Assign keywords for each topic

In [222]:
df = pd.read_csv("./csv/df_translated_tweets.csv")

In [223]:
def clean_translated_tweets(tweet):
    if isinstance(tweet, float):
        return 'NaN'
    else:
        return tweet[23:-3]

In [224]:
df['translated_tweets'] = df['translated_tweets'].apply(clean_translated_tweets)

In [225]:
df = df[df.translated_tweets != "NaN"]

In [226]:
df = df.reset_index(drop=True)

In [227]:
df = df.drop('Unnamed: 0', axis=1)

In [228]:
df.to_csv('./csv/df_translated_tweets_clean.csv')

In [209]:
topics_to_keyword = {}
topics_to_keyword['family'] = {'pension', 'AHV', 'AVS', 'benefit', 'benefits', 'childcare', 'nursery', 'paternity', 'pension', 'life expectancy', 'social assistance', 'social housing', 'construction', 'public housing'}
topics_to_keyword['health'] = {'health', 'premium', 'organ', 'donation', 'vaccine', 'vaccination', 'insurance', 'premium', 'subsidies', 'cost brake', 'health care cost'}
topics_to_keyword['education'] = {'education', 'courses', 'subsidies', 'religion', 'sex education', 'reconversion', 'learning', 'school', 'highschool', 'apprenticeship', 'disabilities'}
topics_to_keyword['immigration'] = {'resident', 'communal elections', 'bilateral', 'EU', 'immigration', 'foreigners', 'undocumented', 'residence', 'migrants', 'asylum', 'naturalisation', 'integration'}
topics_to_keyword['finances'] = {'taxes', 'financial equalisation', 'individual taxation', 'married', 'company taxation'}
topics_to_keyword['economy'] = {'electricity', 'wage', 'FDI', 'investment controls', 'takeovers', 'opening hours', 'shops', 'dismissal', 'workers', ' public services', 'post office'}
topics_to_keyword['digitalisation'] = {'Uber', 'Airbnb', 'online'}
topics_to_keyword['energy'] = {'fossil', 'fuel', 'C02', 'gas', 'renewable', 'energy', 'motorway', 'lanes', 'road pricing'}
topics_to_keyword['ecology'] = {'predators', 'lynx', 'bears', 'wolf', 'plants', 'genetically modified', 'ecologica', 'farms', 'pesticides', 'biodiversity', 'animals', 'livestock'}
topics_to_keyword['politics'] = {'campaigns', 'transparency', 'e-voting', 'votes', 'voting age'}
topics_to_keyword['security'] = {'border control', 'civil service', 'schengen', 'shengen', 'war', 'weapons', 'army', 'aircraft', 'dna'}
topics_to_keyword['external_relations'] = {'EU', 'trade', 'USA', 'human rights', 'environment', 'security council'}




## Classify the tweets

In [210]:

classes = [set() for _ in range(len(df))]

for i,tweet in enumerate(df["translated_tweets"]):
    for topic in topics_to_keyword:
        for keyword in topics_to_keyword[topic]:
            if keyword in tweet:
                classes[i].add(topic)
        

In [217]:
df["topic"] = classes

## how many tweets not assigned to a topic?

In [218]:
res = 0
for topic in df['topic']:
    if len(topic) == 0:
        res += 1

print(res)
# many sentences without topic

1138


In [219]:
df

,tweets,likes,time,politician_id,language,translated_tweets,topic
0,Alors que le Conseil fédéral annonce des défic...,5,2022-09-14 08:08:53+00:00,CarloSommaruga,"((0, 253, 'FRENCH', 'fr'),)",While the Federal Council is announcing budget...,{}
1,Les excédents du fonds AVS pour 2020 et 2021 s...,0,2022-08-26 12:28:02+00:00,CarloSommaruga,"((0, 185, 'FRENCH', 'fr'),)",The surplus of the AVS fund for 2020 and 2021 ...,{family}
2,"Es ist pikant, dass ein Fakejournalist, aber e...",4,2022-08-26 12:22:12+00:00,CarloSommaruga,"((0, 244, 'GERMAN', 'de'),)","It is piqued that a fake journalist, but a rea...",{family}
3,Les statuts permettent l’élection directe et p...,2,2022-08-22 05:44:51+00:00,CarloSommaruga,"((0, 288, 'FRENCH', 'fr'),)",The statutes allow for direct election and by ...,{}
4,Les Suisses et Suissesses sont un peuple de lo...,5,2022-08-21 06:23:55+00:00,CarloSommaruga,"((0, 104, 'FRENCH', 'fr'),)","The Swiss are a people of tenants, and they re...",{}
...,...,...,...,...,...,...,...
1605,Comment le premier parti de suisse peut-il pro...,2,2015-12-08 17:13:38+00:00,CarloSommaruga,"((0, 123, 'FRENCH', 'fr'),)",How can the first Swiss party propose so much ...,{}
1606,CH-EU:73% des entreprises craignent que résili...,0,2015-12-08 10:03:12+00:00,CarloSommaruga,"((0, 120, 'FRENCH', 'fr'),)",CH-EU:73% of companies fear that termination o...,"{external_relations, immigration}"
1607,"Suisse: De 2009 à 2014, le nombre de jeunes pr...",0,2015-12-07 08:51:27+00:00,CarloSommaruga,"((0, 119, 'FRENCH', 'fr'),)","Switzerland: From 2009 to 2014, the number of ...",{}
1608,Suisse-UE: Clause de sauvegarde unilatérale: P...,1,2015-12-05 15:58:37+00:00,CarloSommaruga,"((0, 118, 'FRENCH', 'fr'),)",Switzerland-EU: Unilateral safeguard clause: P...,"{external_relations, immigration}"


# Testing Question Answering # 1

In [477]:
question_paternity_leave = "Would you like to introduce a paternity leave of 4weeks? "

## Distilbert Model 

In [478]:
context_easy = "Of course, I am in favor of introducing the paternity leave."

In [479]:
qa_model = pipeline("question-answering")
qa_model(question = question_paternity_leave, context = context_easy)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


{'score': 0.15390625596046448,
 'start': 16,
 'end': 59,
 'answer': 'in favor of introducing the paternity leave'}

In [480]:
context_medium= "A paternity leave of 8 weeks would make sense.  "

In [481]:
qa_model = pipeline("question-answering")
qa_model(question = question_paternity_leave, context = context_medium)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


{'score': 0.2950955331325531,
 'start': 0,
 'end': 22,
 'answer': 'A paternity of 8 weeks'}

In [649]:
context_difficult= "Men who take paternity leave have closer bonds with their kids and a lower chance of divorcing their spouse. Paid leave taken by both parents increases women’s participation in the work force. I am therefore not in favor of this initiative"

In [650]:
qa_model = pipeline("question-answering")
qa_model(question = question_paternity_leave, context = context_difficult)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


{'score': 0.15825620293617249,
 'start': 208,
 'end': 220,
 'answer': 'not in favor'}

## Bert QA

In [485]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("srcocotero/bert-large-qa")

model_qa_bert = AutoModelForQuestionAnswering.from_pretrained("srcocotero/bert-large-qa")

In [651]:
qa_model = pipeline("question-answering", model = "srcocotero/bert-large-qa")
qa_model(question = question_paternity_leave, context = context_difficult)

{'score': 0.17064452171325684,
 'start': 193,
 'end': 239,
 'answer': 'I am therefore not in favor of this initiative'}

In [489]:
qa_model = pipeline("question-answering", model = "srcocotero/bert-large-qa")
qa_model(question = question_paternity_leave, context = context_medium)

{'score': 0.28941914439201355,
 'start': 0,
 'end': 39,
 'answer': 'A paternity of 8 weeks would make sense'}

In [490]:
qa_model = pipeline("question-answering", model = "srcocotero/bert-large-qa")
qa_model(question = question_paternity_leave, context = context_easy)

{'score': 0.3168155252933502,
 'start': 11,
 'end': 24,
 'answer': 'I am in favor'}

## Deepset Roberta Model

In [655]:
model_name = "deepset/xlm-roberta-base-squad2"

In [654]:
# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': question_paternity_leave,
    'context': context_easy
}
res = nlp(QA_input)

# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

res

{'score': 0.12847624719142914,
 'start': 10,
 'end': 60,
 'answer': ' I am in favor of introducing the paternity leave.'}

In [656]:
# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': question_paternity_leave,
    'context': context_medium
}
res = nlp(QA_input)

# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

res

{'score': 0.0024764190893620253, 'start': 14, 'end': 22, 'answer': ' 8 weeks'}

In [418]:
# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': question_paternity_leave,
    'context': context_difficult
}
res = nlp(QA_input)

# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

res

{'score': 0.0004850009281653911,
 'start': 108,
 'end': 141,
 'answer': ' Paid leave taken by both parents'}

# Application on the tweets with topic 'family'

In [88]:
#questions for the topic 'family '
questions_family = ["Êtes-vous favorable à une hausse de l'âge de la retraite?", 
                    "L'État devrait-il renforcer son soutien financier à la création de structures d'accueil extra-familial?", 
                    "Une initiative demande l'instauration d'un congé paternité payé de quatre semaines. Êtes-vous en faveur de cette proposition?", 
                    "Les rentes de la caisse de pension doivent-elles être adaptées à l'allongement de l'espérance de vie (réduction du taux de conversion)?", 
                    "Soutenez-vous les efforts au niveau des cantons visant à réduire les prestations d'aide sociale?", 
                   "La Confédération devrait-elle encourager davantage la construction de logements d'utilité publique?"]

question_family_simplified = ['Are you in favour of raising the retirement age?', 
                              'Should the state finance childcare?', 
                              'Is paternity leave good?', 
                              'Should we finance pension funds?', 
                              'Are you in favor of benefits?', 
                              'Are you in favor of public housing?']


In [25]:
#translate the questions
questions_family_trans = []
for question in questions_family:
    questions_family_trans.append(translator_fr(question))

In [31]:
#find the tweets with keywords that relates to the family topic
context_tweets = []
for i in range(len(df)):
    if 'family' in df.iloc[i]['topic']:
        context_tweets.append(df.iloc[i]['translated_tweets'][0]['translation_text'])

#join all the context tweets
context_tweets_all = ' '.join(context_tweets)        

In [165]:
# this is our context for the QA with topic family
context_tweets_all

'The surplus of the AVS fund for 2020 and 2021 is a true fact which is at p.76 of the Management Report 2021 of the AVS Fund! Don\'t let yourself be influenced by a fakejournalist! It is piqued that a fake journalist, but a real ideologist, nevertheless denies a true fact, which can be found on page 76 of the Annual Report 2021 of the AHV Fund! Therefore a NO to the fake journalists and a 2 x NO to the AHV 21. The purchasing power of many people is under increasing pressure: wages and pensions are stagnating, but energy prices, rents and health insurance premiums continue to rise. The socially irresponsible attitude of the PLR, Centre and UDC will place many tenants at the AVS or the AI in a very precarious financial situation with the risk of termination of the contract if the amounts are not paid. The Swiss National Bank (SNB) collects billions of profits. This money belongs to the population and must finally come back to us. This is precisely what the SNB initiative wants: with it, 

In [ ]:
#create with tweets all


### First Question: 'Are you in favour of raising the retirement age?'

In [51]:
#first question
questions_family_trans[0][0]['translation_text']

'Are you in favour of raising the retirement age?'

In [33]:
qa_model = pipeline("question-answering", model = "srcocotero/bert-large-qa")
qa_model(question = questions_family_trans[0][0]['translation_text'], context = context_tweets_all)

Downloading:   0%|          | 0.00/675 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/695k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


{'score': 0.3342711627483368, 'start': 3426, 'end': 3433, 'answer': 'against'}

In [37]:
context_tweets_all[3300:3600]

'Confederation, be paid into a continuing education and vocational integration fund. For worthy pensions, we commit ourselves: against the rise in the retirement age of women for the increase of AVS pensions and the sustainability of their funding for the adequate and quality care of the elderly We a'

In [68]:
#using simplified question
question_family_simplified[0]

'Are you in favour of raising the retirement age?'

In [67]:
qa_model = pipeline("question-answering", model = "srcocotero/bert-large-qa")
qa_model(question = question_family_simplified[0], context = context_tweets_all)

{'score': 0.3342711627483368, 'start': 3426, 'end': 3433, 'answer': 'against'}

### Second Question: 'Should the State increase its financial support for the creation of extra-family reception facilities?'

In [52]:
#second question
questions_family_trans[1][0]['translation_text']

'Should the State increase its financial support for the creation of extra-family reception facilities?'

In [38]:
qa_model(question = questions_family_trans[1][0]['translation_text'], context = context_tweets_all)

{'score': 0.06996136158704758, 'start': 8044, 'end': 8045, 'answer': '.'}

In [83]:
#using simplified question
question_family_simplified[1]

'Should the state finance childcare?'

In [84]:
qa_model = pipeline("question-answering", model = "srcocotero/bert-large-qa")
qa_model(question = question_family_simplified[1], context = context_tweets_all)

{'score': 0.09597817808389664,
 'start': 8265,
 'end': 8300,
 'answer': 'the financing of the AVS is assured'}

### Third Question: 'One initiative calls for a four-week paid paternity leave. Are you in favour of this proposal?'

In [56]:
questions_family_trans[2][0]['translation_text']

'One initiative calls for a four-week paid paternity leave. Are you in favour of this proposal?'

In [39]:
qa_model(question = questions_family_trans[2][0]['translation_text'], context = context_tweets_all)

{'score': 0.23817914724349976,
 'start': 2282,
 'end': 2290,
 'answer': 'approval'}

In [44]:
context_tweets_all[2200:2400]

'y of the people and of the opening are the with refusal of subsidies to the rich, approval of paternity leave and maintenance of the bilateral way and the success of the minimum wage and the defence o'

In [87]:
#using simplified question
question_family_simplified[2]

'Is paternity leave good?'

In [72]:
qa_model = pipeline("question-answering", model = "srcocotero/bert-large-qa")
qa_model(question = question_family_simplified[2], context = context_tweets_all)

{'score': 0.21026499569416046,
 'start': 2282,
 'end': 2290,
 'answer': 'approval'}

### Fourth Question: 'One initiative calls for a four-week paid paternity leave. Are you in favour of this proposal?'

In [54]:
questions_family_trans[3][0]['translation_text']

'One initiative calls for a four-week paid paternity leave. Are you in favour of this proposal?'

In [45]:
qa_model(question = questions_family_trans[3][0]['translation_text'], context = context_tweets_all)

{'score': 0.04313672333955765, 'start': 8044, 'end': 8045, 'answer': '.'}

In [89]:
#using simplified question
question_family_simplified[3]

'Should we finance pension funds?'

In [90]:
qa_model = pipeline("question-answering", model = "srcocotero/bert-large-qa")
qa_model(question = question_family_simplified[3], context = context_tweets_all)

{'score': 0.21115420758724213, 'start': 5247, 'end': 5254, 'answer': 'against'}

### Fifth Question: 'Do you support cantonal efforts to reduce social assistance benefits?'

In [57]:
questions_family_trans[4][0]['translation_text']

'Do you support cantonal efforts to reduce social assistance benefits?'

In [46]:
qa_model(question = questions_family_trans[4][0]['translation_text'], context = context_tweets_all)

{'score': 0.04994482547044754,
 'start': 6678,
 'end': 6748,
 'answer': 'My total support for the DEAS for its action against fraudulent clinic'}

In [47]:
context_tweets_all[6500:6900]

"eneva's families. In the cantonal and federal parliament, the elected representatives of Geneva and Geneva are committed to these themes for the benefit of the people of Geneva. My total support for the DEAS for its action against fraudulent clinic! How many private insured people abusively receive benefits!The DEAS must require the OCAI to put an end to any collaboration Parliamentary news: Atten"

In [75]:
#using simplified question
question_family_simplified[4]

'Are you in favor of benefits?'

In [76]:
qa_model = pipeline("question-answering", model = "srcocotero/bert-large-qa")
qa_model(question = question_family_simplified[4], context = context_tweets_all)

{'score': 0.08064859360456467,
 'start': 1851,
 'end': 1919,
 'answer': 'the states have voted in favour of the retirement of women at age 65'}

### Sixth Question: 'Should Confederation further encourage the construction of public utility housing?'


In [ ]:
questions_family_trans[5][0]['translation_text']

In [48]:
qa_model(question = questions_family_trans[5][0]['translation_text'], context = context_tweets_all)

{'score': 0.1102256253361702,
 'start': 2904,
 'end': 2950,
 'answer': "It's time to focus on AVS + affordable housing"}

In [60]:
context_tweets_all[2700:3200]

"evant. But the reserve for future distributions in the Confederation and the Cantons. End 2019: 86 Mia frs! It's more than 20 years of distribution. The BNS wants to negotiate an additional distribution. It's time to focus on AVS + affordable housing With the ageing of the population, guaranteeing worthy pensions to current/future pensioners is essential. We commit ourselves: against the rise of the retirement age of women for the increase of AVS pensions and the sustainability of their financin"

In [78]:
#using simplified question
question_family_simplified[5]

'Are you in favor of public housing?'

In [79]:
qa_model = pipeline("question-answering", model = "srcocotero/bert-large-qa")
qa_model(question = question_family_simplified[5], context = context_tweets_all)

{'score': 0.07114294916391373,
 'start': 4455,
 'end': 4476,
 'answer': 'we are not against it'}

In [80]:
context_tweets_all[4000:5000]

'leave Support for nurseries Best pensions for women With the same conviction against all types of discrimination Indecisive about tax reform and the reform of the AVS ? I am debating this evening at 6.30 p.m. (room MR070) under moderation For affordable housing, an open house or dignified pensions, what are our commitments for Geneva? Discover them here The call for public housing policies to be excluded from the scope of the framework agreement, and we are not against it, but we would like to make the voice of the tenants heard in this debate. It is time to commit ourselves to making our country the cradle of responsible multinationals, a challenge that will benefit everyone. Sign the announcement Very happy that the Geneva economic fabric is doing so well. It is now necessary that these successes benefit all and not some, you agree? Geneva The SBBs have received many free land for railway operations. Converted to housing construction, they must be allocated to affordable housing rat

## Model huggingface-course/bert-finetuned-squad

In [553]:
qa_model = pipeline("question-answering", model = "huggingface-course/bert-finetuned-squad")
qa_model(question = question_paternity_leave, context = context_difficult)

Downloading:   0%|          | 0.00/686 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/320 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

{'score': 0.13527600467205048,
 'start': 193,
 'end': 235,
 'answer': 'I am therefore in favor of this initiative'}

# Testing Question Answering # 1

In [ ]:
question_income_tax = 

In [226]:
question_translated

'Are you in favour of raising retirement age?'

In [227]:
qa_model = pipeline("question-answering")
qa_model(question = question_translated, context = paragraph, model = "deepset/roberta-base-squad2")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)
/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


{'score': 0.03753843903541565,
 'start': 1819,
 'end': 1849,
 'answer': 'Let every activist participate'}

In [142]:
res

{'score': 0.48647525906562805,
 'start': 261,
 'end': 325,
 'answer': 'states voted in favour of retirement of women at 65 years of age'}

In [ ]:
#1. drop rows when empty
#2. create 2 questions, and create 3 contexts, try question answering
#--> investigate question answering


#3. classify tweets, and add category in dataset
# later step: match tweets category with question category to create the context for questions answering (huggingface)


In [115]:
df_questions = pd.read_csv('./csv/carlo_questions_keywords.csv', sep=';', encoding='latin-1')

In [130]:
df_questions

,question,smartvote_answers,binary_smarvote_answer,huggingface_answers,cami_answers,Topic,Keywords,Simplified question
0,Êtes-vous favorable à une hausse de l'âge de l...,4,NaN,NaN,NaN,Family,pension; AHV; AVS,''Are you in favour of raising the retirement ...
1,L'État devrait-il renforcer son soutien finan...,1,NaN,NaN,NaN,Family,benefit; childcare; nursery,''Should the state support childcare? ''
2,Une initiative demande l'instauration d'un co...,1,NaN,NaN,NaN,Family,paternity; leave,''Are you in favour of a paternity leave?''
3,Les rentes de la caisse de pension doivent-el...,4,NaN,NaN,NaN,Family,pension; life expectancy,''Should the finance state pension funds?''
4,Soutenez-vous les efforts au niveau des canto...,4,NaN,NaN,NaN,Family,benefits; social assistance,''Are you in favor of benefits?''
5,La Confédération devrait-elle encourager dava...,1,NaN,NaN,NaN,Family,social housing; construction; public housing,''Are you in favor of public housing?''
6,Les personnes assurées devraient-elles contri...,4,NaN,NaN,NaN,Health,health cost; health care cost; premium,''Should the health insurance cost more''
7,Approuveriez-vous le principe du consentement...,1,NaN,NaN,NaN,Health,organ; donation,''Is organ donation good?''
8,Une vaccination obligatoire des enfants basée ...,1,NaN,NaN,NaN,Health,vaccine; vaccination,''Should vaccines be compulsory? ''
9,Une initiative demande que les subsides aux a...,1,NaN,NaN,NaN,Health,insurance; premium; subsidies,''Should health be proportional to your income?''


In [ ]:
#remove numeration
#if loop for binary answers
#run huggingface answers


In [125]:
df_questions["question"] = df_questions['question'].str[4:]
#guillemet

In [149]:
df_questions.loc[df_questions['smartvote_answers'] >= 3, 'binary_smarvote_answer'] = 0

In [150]:
df_questions.loc[df_questions['smartvote_answers'] <= 2, 'binary_smarvote_answer'] = 1

In [151]:
df_questions

,question,smartvote_answers,binary_smarvote_answer,huggingface_answers,cami_answers,Topic,Keywords,Simplified question
0,Êtes-vous favorable à une hausse de l'âge de l...,4,0.0,NaN,NaN,Family,pension; AHV; AVS,''Are you in favour of raising the retirement ...
1,L'État devrait-il renforcer son soutien finan...,1,1.0,NaN,NaN,Family,benefit; childcare; nursery,''Should the state support childcare? ''
2,Une initiative demande l'instauration d'un co...,1,1.0,NaN,NaN,Family,paternity; leave,''Are you in favour of a paternity leave?''
3,Les rentes de la caisse de pension doivent-el...,4,0.0,NaN,NaN,Family,pension; life expectancy,''Should the finance state pension funds?''
4,Soutenez-vous les efforts au niveau des canto...,4,0.0,NaN,NaN,Family,benefits; social assistance,''Are you in favor of benefits?''
5,La Confédération devrait-elle encourager dava...,1,1.0,NaN,NaN,Family,social housing; construction; public housing,''Are you in favor of public housing?''
6,Les personnes assurées devraient-elles contri...,4,0.0,NaN,NaN,Health,health cost; health care cost; premium,''Should the health insurance cost more''
7,Approuveriez-vous le principe du consentement...,1,1.0,NaN,NaN,Health,organ; donation,''Is organ donation good?''
8,Une vaccination obligatoire des enfants basée ...,1,1.0,NaN,NaN,Health,vaccine; vaccination,''Should vaccines be compulsory? ''
9,Une initiative demande que les subsides aux a...,1,1.0,NaN,NaN,Health,insurance; premium; subsidies,''Should health be proportional to your income?''


In [147]:
qa_model = pipeline("question-answering", model = "srcocotero/bert-large-qa")

def qa(question_smartvote):
    return qa_model(question = question_smartvote , context = context_tweets_all)

In [152]:
qa(question_smartvote = "Are you in favor of gay marriage")

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


{'score': 0.10621041804552078, 'start': 1119, 'end': 1120, 'answer': '.'}

In [148]:
df_questions["huggingface_answers"] = df["question"].apply(qa)


KeyError: 'question'